In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scanpy as sc
from read_data import *
import csv
import math

In [ ]:
#sample_names = ['76-2b','76-2f','79-1d','79-1e','79-2e','79-2f']
sample_name = '79-2f'
src = f'/cellranger/{sample_name}' # path to raw cellranger files
# organize data into anndata object
dat_arr = read_data(src) 
adata = make_anndata(dat_arr)

In [ ]:
sc.settings.set_figure_params(dpi=80,dpi_save=600,facecolor='white',format='png')

In [ ]:
# filter genes ocurring in less than 3 cells
min_cells_per_gene = 3
sc.pp.filter_genes(adata, min_cells=min_cells_per_gene)

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

In [ ]:
adata.obs['log10_n_genes_by_counts'] = np.log10(adata.obs.n_genes_by_counts)
adata.obs['log10_total_counts'] = np.log10(adata.obs.total_counts)

In [ ]:
plt.figure(figsize=(5,5))
plt.hist(adata.obs["log10_n_genes_by_counts"], bins=50, alpha=0.5,ec='black')
plt.axvline(adata.obs["log10_n_genes_by_counts"].median(), color='k', linestyle='dashed', linewidth=1)
plt.ylabel('Frequency')
plt.xlabel("log10_n_genes_by_counts")
ax = plt.gca()
plt.grid(visible=None)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.hist(adata.obs["log10_total_counts"], bins=50, alpha=0.5,ec='black')
plt.axvline(adata.obs["log10_total_counts"].median(), color='k', linestyle='dashed', linewidth=1)
plt.ylabel('Frequency')
plt.xlabel("log10_total_counts")
ax = plt.gca()
plt.grid(visible=None)
plt.show()

In [ ]:
adata.var['mt'] = adata.var.gene_name.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata.obs['log10_total_counts'] = np.log10(adata.obs.total_counts)
adata.obs['log10_n_genes_by_counts'] = np.log10(adata.obs.n_genes_by_counts)
adata.var['log10_total_counts'] = np.log10(adata.var.total_counts)

In [ ]:
sc.pl.violin(adata,['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],scale='area',
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
# filter genes per cell and umis per cell based on top and bottom percentiles
# filter cells based on percentage of mitochondrial genes
low_genes_per_cell_percentile = 5
high_genes_per_cell_percentile = 99
low_counts_per_cell_percentile = 5
high_counts_per_cell_percentile = 99
min_genes_per_cell = round(np.percentile(adata.obs.n_genes_by_counts,low_genes_per_cell_percentile))
print('min genes per cell: '+str(min_genes_per_cell))
max_genes_per_cell = round(np.percentile(adata.obs.n_genes_by_counts,high_genes_per_cell_percentile))
print('max genes per cell: '+str(max_genes_per_cell))
min_counts_per_cell = round(np.percentile(adata.obs.total_counts,low_genes_per_cell_percentile))
print('min counts per cell: '+str(min_counts_per_cell))
max_counts_per_cell = round(np.percentile(adata.obs.total_counts,high_genes_per_cell_percentile))
print('max counts per cell: '+str(max_counts_per_cell))

In [ ]:
min_genes_bool=sc.pp.filter_cells(adata, min_genes=min_genes_per_cell,inplace=False)
max_genes_bool=sc.pp.filter_cells(adata, max_genes=max_genes_per_cell,inplace=False)
min_counts_bool=sc.pp.filter_cells(adata, min_counts=min_counts_per_cell,inplace=False)
max_counts_bool=sc.pp.filter_cells(adata, max_counts=max_counts_per_cell,inplace=False)
min_genes_bool = [not x for x in min_genes_bool[0]]
max_genes_bool = [not x for x in max_genes_bool[0]]
min_counts_bool = [not x for x in min_counts_bool[0]]
max_counts_bool = [not x for x in max_counts_bool[0]]

In [ ]:
thresholded = [a + b + c + d for a, b, c, d in zip(min_genes_bool,max_genes_bool,min_counts_bool,max_counts_bool)]
thresholded = [bool(x) for x in thresholded]

In [ ]:
sum(thresholded)

In [ ]:
thresholded_x = adata.obs.total_counts[thresholded]
thresholded_y = adata.obs.n_genes_by_counts[thresholded]
plt.scatter(adata.obs.total_counts,adata.obs.n_genes_by_counts,s=1,c='dodgerblue')
plt.scatter(thresholded_x,thresholded_y,c='r',s=1)
plt.xlabel('total_counts')
plt.ylabel('pct_counts_mt')
colors=['dodgerblue','r']
texts=['maintained cells','filtered cells']
patches = [ plt.plot([],[], marker="o", ms=3, ls="", linewidth=0.6,color=colors[i], 
                label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, bbox_to_anchor=(1, 0.5), 
        loc='center left', ncol=1,fontsize='medium')

In [ ]:
max_mt_pct = 5
max_mt_bool = list(adata.obs.pct_counts_mt >= max_mt_pct)
thresholded_x = adata.obs.total_counts[max_mt_bool]
thresholded_y = adata.obs.pct_counts_mt[max_mt_bool]
plt.scatter(adata.obs.total_counts,adata.obs.pct_counts_mt,s=1,c='dodgerblue')
plt.scatter(thresholded_x,thresholded_y,c='r',s=1)
plt.xlabel('total_counts')
plt.ylabel('pct_counts_mt')
colors=['dodgerblue','r']
texts=['maintained cells','filtered cells']
patches = [ plt.plot([],[], marker="o", ms=3, ls="", linewidth=0.6,color=colors[i], 
                label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, bbox_to_anchor=(1, 0.5), 
        loc='center left', ncol=1,fontsize='medium')

In [ ]:
min_genes_bool=sc.pp.filter_cells(adata, min_genes=min_genes_per_cell,inplace=True)
max_genes_bool=sc.pp.filter_cells(adata, max_genes=max_genes_per_cell,inplace=True)
min_counts_bool=sc.pp.filter_cells(adata, min_counts=min_counts_per_cell,inplace=True)
max_counts_bool=sc.pp.filter_cells(adata, max_counts=max_counts_per_cell,inplace=True)
adata = adata[adata.obs.pct_counts_mt < max_mt_pct, :]

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
plt.figure(figsize=(5,5))
plt.hist(adata.obs["log10_total_counts"], bins=50, alpha=0.5,ec='black')
plt.axvline(adata.obs["log10_total_counts"].median(), color='k', linestyle='dashed', linewidth=1)
plt.ylabel('Frequency')
plt.xlabel("log10_total_counts")
ax = plt.gca()
plt.grid(visible=None)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.hist(adata.obs["log10_n_genes_by_counts"], bins=50, alpha=0.5,ec='black')
plt.axvline(adata.obs["log10_n_genes_by_counts"].median(), color='k', linestyle='dashed', linewidth=1)
plt.ylabel('Frequency')
plt.xlabel("log10_n_genes_by_counts")
ax = plt.gca()
plt.grid(visible=None)
plt.show()

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(adata, ['log10_n_genes_by_counts', 'log10_total_counts'],
             jitter=0.4, multi_panel=True)

In [ ]:
out_path = f'{sample_name}_cell_gene_filtration_params.csv'
a_file = open(out_path, "w")
gene_cell_filtration_params={
    'low_genes_per_cell_percentile': low_genes_per_cell_percentile,
    'min_genes_per_cell': min_genes_per_cell,
    'high_genes_per_cell_percentile': high_genes_per_cell_percentile,
    'max_genes_per_cell': max_genes_per_cell,
    'low_counts_per_cell_percentile': low_counts_per_cell_percentile,
    'min_counts_per_cell': min_counts_per_cell,
    'high_counts_per_cell_percentile': high_counts_per_cell_percentile,
    'max_counts_per_cell': max_counts_per_cell,
    'min_cells_per_gene': min_cells_per_gene,
    'max_mt_pct': max_mt_pct
}
writer = csv.writer(a_file)
for key, value in gene_cell_filtration_params.items():
    writer.writerow([key, value])
a_file.close()

In [ ]:
out_path = f'{sample_name}_filtered_adata.h5ad'
adata.write(out_path)